In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import uniform
from scipy.stats import randint
# insert other models you want to use here
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# replace filepath here
data = pd.read_csv('/content/drive/MyDrive/Fraud project 3/after_feature_selection_forward_10.csv', parse_dates=['Date'])
data.head()

,total_Amount_card_description_7,total_Amount_card_description_3,max_Amount_card_state_14,total_Amount_Cardnum_3,total_Amount_card_description_1,total_Amount_Cardnum_0,total_Amount_card_description_0,total_Amount_Cardnum_14,max_Amount_card_state_30,max_Amount_Cardnum_3,Date,Fraud,Amount
0,293.22,293.22,3.67,438.57,293.22,438.57,293.22,438.57,3.67,3.67,2010-01-01,0,3.62
1,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,2010-01-01,0,31.42
2,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,2010-01-01,0,178.49
3,93.20,93.20,3.80,182.84,93.20,182.84,93.20,182.84,3.80,3.80,2010-01-01,0,3.62
4,293.22,293.22,3.67,438.57,293.22,438.57,293.22,438.57,3.67,3.67,2010-01-01,0,3.62


In [ ]:
train_test = data[(data.Date >= '2010-01-15') & (data.Date <= '2010-10-31')].drop(columns=['Date'])
OOT = data[data.Date > '2010-10-31'].drop(columns=['Date'])

In [ ]:
X = train_test.drop(columns=['Fraud', 'Amount'])
y = train_test['Fraud']
X_OOT = OOT.drop(columns=['Fraud', 'Amount'])
y_OOT = OOT['Fraud']

In [ ]:
def get_fdr(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    temp = pd.DataFrame({'y': y, 'y_pred': y_pred})
    actual_pos = y.sum()
    temp = temp.sort_values('y_pred', ascending=False)
    target_num = int(temp.shape[0] * 0.03)
    temp = temp.iloc[:target_num, :]
    detect_pos = temp['y'].sum()
    fdr = detect_pos / actual_pos
    return fdr

### Randomized Search

In [ ]:
model = SVC(probability=True)
model.fit(X, y)
predictions = model.predict(X_OOT)


In [ ]:
print(classification_report(y_OOT, predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     12248
           1       0.18      0.11      0.13       179

    accuracy                           0.98     12427
   macro avg       0.58      0.55      0.56     12427
weighted avg       0.98      0.98      0.98     12427



In [ ]:
model.predict_proba(X_OOT)

array([[0.99351408, 0.00648592],
       [0.99350033, 0.00649967],
       [0.9935207 , 0.0064793 ],
       ...,
       [0.9935128 , 0.0064872 ],
       [0.9935179 , 0.0064821 ],
       [0.99348474, 0.00651526]])

In [ ]:
get_fdr(model,X_OOT,y_OOT)

0.4022346368715084

In [ ]:
model._gamma

5.4140542222494136e-12

In [ ]:
# Example, use your model and hyperparameters
lgb = SVC(probability=True)

parameters = dict(C=uniform(loc=0.5, scale=1),
                     gamma=uniform(loc=0.1/(np.array(X).var()*X.shape[1]), scale=10/(np.array(X).var()*X.shape[1])),
                    kernel=['rbf','poly','sigmoid'])

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
searcher_lgb = RandomizedSearchCV(lgb, parameters, scoring=get_fdr, n_jobs=-1, random_state=0)
searcher_lgb.fit(X_scaled, y)

print(searcher_lgb.best_params_)
print(searcher_lgb.best_score_)

{'C': 1.0820197920751071, 'gamma': 2.9635083440447154e-11, 'kernel': 'rbf'}
0.6554248887117134


In [ ]:
params = searcher_lgb.best_params_

In [ ]:
params

{'C': 1.0820197920751071, 'gamma': 2.9635083440447154e-11, 'kernel': 'rbf'}

In [ ]:
params['probability'] = True
params

{'C': 1.0820197920751071,
 'gamma': 2.9635083440447154e-11,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_1={'C': 1.0820197920751071,
 'gamma': 2.9635083440447154e-11,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_2={'C': 0.1,
 'gamma': 2.9635083440447154e-10,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_3={'C': 0.1,
 'gamma': 2.9635083440447154e-11,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_4={'C': 10,
 'gamma': 2.9635083440447154e-11,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_5={'C': 10,
 'gamma': 2.9635083440447154e-10,
 'kernel': 'rbf',
 'probability': True}

In [ ]:
params_list=[params_4,params_5]

### Modeling

In [ ]:
df_list=[]
for params in params_list:
  # Whether you want to resample or not
  smote_flag = True

  fdr_scores = pd.DataFrame(index=range(10), columns=['train fdr', 'test fdr', 'oot fdr'])
  for i in range(10):
      # Use your own model and hyperparameters here
      model = SVC(**params)
      
      X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
      scaler = StandardScaler()
      X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
      X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
      X_OOT_scaled = pd.DataFrame(scaler.transform(X_OOT), columns=X_OOT.columns)
      
      #Capped 6
      drop_index = ((X_train_scaled >= 6) | (X_train_scaled <= -6)).sum(axis=1)
      drop_index = drop_index[drop_index >= 1].index.to_list()
      X_train_scaled = X_train_scaled.drop(index=drop_index)
      y_train = pd.DataFrame(y_train).iloc[X_train_scaled.index, :]['Fraud']
      
      if smote_flag:
          y_neg_cnt = (y_train == 0).sum()
          smote = SMOTE(sampling_strategy={0: y_neg_cnt, 1: int(y_neg_cnt/10)})
          X_train_smo, y_train_smo = smote.fit_sample(X_train_scaled, y_train)
          model.fit(X_train_smo, y_train_smo)
      else:
          model.fit(X_train_scaled, y_train)
          
      fdr_scores.loc[i, 'train fdr'] = get_fdr(model, X_train_scaled, y_train)
      fdr_scores.loc[i, 'test fdr'] = get_fdr(model, X_test_scaled, y_test)
      fdr_scores.loc[i, 'oot fdr'] = get_fdr(model, X_OOT_scaled, y_OOT)
  print(params,' done')
  print(fdr_scores)
  df_list.append(fdr_scores)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

{'C': 10, 'gamma': 2.9635083440447154e-11, 'kernel': 'rbf', 'probability': True}  done
  train fdr  test fdr   oot fdr
0  0.521739  0.686636  0.458101
1  0.525751  0.631336  0.458101
2  0.549474  0.612903  0.458101
3  0.540541  0.640553  0.463687
4  0.534021  0.672811  0.463687
5  0.529412  0.654378  0.458101
6  0.536383   0.64977  0.458101
7  0.554852  0.599078  0.458101
8  0.534591   0.64977  0.458101
9  0.531381  0.658986  0.458101


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

{'C': 10, 'gamma': 2.9635083440447154e-10, 'kernel': 'rbf', 'probability': True}  done
  train fdr  test fdr   oot fdr
0  0.544885  0.626728  0.458101
1  0.540881  0.631336  0.458101
2  0.514768  0.746544  0.452514
3  0.523109  0.677419  0.458101
4  0.508811  0.654378  0.458101
5  0.541152  0.654378  0.458101
6   0.57173  0.631336  0.452514
7  0.552301  0.612903  0.458101
8   0.53112  0.682028  0.458101
9  0.535714  0.640553  0.458101


In [ ]:
from google.colab import files

In [ ]:
j=0
for i in df_list:
  fname=f'df_{j}.csv'
  i.to_csv(fname,index=False)
  files.download(fname)
  j+=1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for i in df_list:
  print(i['train fdr'].mean(),i['test fdr'].mean(),i['oot fdr'].mean())
  

0.5358143616574076 0.6456221198156682 0.4592178770949721
0.5364471523546848 0.6557603686635944 0.45698324022346365
